## Deliverable 2. Create a Customer Travel Destinations Map.

In [47]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [48]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Lagoa,PT,39.0500,-27.9833,58.53,70,50,26.51,scattered clouds
1,1,Arraial Do Cabo,BR,-22.9661,-42.0278,78.66,89,75,8.05,broken clouds
2,2,Areka,ET,7.0667,37.7000,62.06,38,90,5.79,overcast clouds
3,3,Touros,BR,-5.1989,-35.4608,83.30,76,20,12.57,few clouds
4,4,Palmer,US,42.1584,-72.3287,22.37,31,20,3.00,few clouds


In [49]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum allowable temperature for your vacation? "))
max_temp = float(input("What is the maximum allowable temperature for your vacation? "))

What is the minimum allowable temperature for your vacation? 65
What is the maximum allowable temperature for your vacation? 95


In [51]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
filtered_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Arraial Do Cabo,BR,-22.9661,-42.0278,78.66,89,75,8.05,broken clouds
3,3,Touros,BR,-5.1989,-35.4608,83.30,76,20,12.57,few clouds
6,6,Ullal,IN,12.8000,74.8500,77.95,78,0,0.00,mist
7,7,Port Macquarie,AU,-31.4333,152.9167,75.88,77,8,4.59,clear sky
9,9,Hobart,AU,-42.8794,147.3294,67.12,86,75,5.75,broken clouds
11,11,Kapaa,US,22.0752,-159.3190,80.58,70,0,1.01,clear sky
12,12,Sao Filipe,CV,14.8961,-24.4956,70.99,74,99,14.36,overcast clouds
14,14,Vaini,TO,-21.2000,-175.2000,80.15,77,100,12.71,light rain
17,17,Aripuana,BR,-9.1667,-60.6333,74.44,98,100,1.83,moderate rain
19,19,Ponta Do Sol,PT,32.6667,-17.1000,65.03,73,1,9.37,clear sky


In [52]:
# 4a. Determine if there are any empty rows.
filtered_cities_df.notnull().sum()

City_ID                326
City                   326
Country                326
Lat                    326
Lng                    326
Max Temp               326
Humidity               326
Cloudiness             326
Wind Speed             326
Current Description    326
dtype: int64

In [53]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
filtered_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Arraial Do Cabo,BR,-22.9661,-42.0278,78.66,89,75,8.05,broken clouds
3,3,Touros,BR,-5.1989,-35.4608,83.30,76,20,12.57,few clouds
6,6,Ullal,IN,12.8000,74.8500,77.95,78,0,0.00,mist
7,7,Port Macquarie,AU,-31.4333,152.9167,75.88,77,8,4.59,clear sky
9,9,Hobart,AU,-42.8794,147.3294,67.12,86,75,5.75,broken clouds


In [54]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = filtered_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Arraial Do Cabo,BR,78.66,broken clouds,-22.9661,-42.0278,
3,Touros,BR,83.30,few clouds,-5.1989,-35.4608,
6,Ullal,IN,77.95,mist,12.8000,74.8500,
7,Port Macquarie,AU,75.88,clear sky,-31.4333,152.9167,
9,Hobart,AU,67.12,broken clouds,-42.8794,147.3294,
11,Kapaa,US,80.58,clear sky,22.0752,-159.3190,
12,Sao Filipe,CV,70.99,overcast clouds,14.8961,-24.4956,
14,Vaini,TO,80.15,light rain,-21.2000,-175.2000,
17,Aripuana,BR,74.44,moderate rain,-9.1667,-60.6333,
19,Ponta Do Sol,PT,65.03,clear sky,32.6667,-17.1000,


In [55]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, city in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = city["Lat"]
    lng = city["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [56]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np

clean_hotel_df = hotel_df
clean_hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df.dropna(subset=['Hotel Name'], inplace=True)

In [57]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Arraial Do Cabo,BR,78.66,broken clouds,-22.9661,-42.0278,Pousada Porto Praia
3,Touros,BR,83.30,few clouds,-5.1989,-35.4608,INN NEW HORIZON
6,Ullal,IN,77.95,mist,12.8000,74.8500,AK INTERNATIONAL LODGE
7,Port Macquarie,AU,75.88,clear sky,-31.4333,152.9167,Rydges Port Macquarie
9,Hobart,AU,67.12,broken clouds,-42.8794,147.3294,Mantra on Collins Hobart
11,Kapaa,US,80.58,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
12,Sao Filipe,CV,70.99,overcast clouds,14.8961,-24.4956,Hotel Xaguate
14,Vaini,TO,80.15,light rain,-21.2000,-175.2000,Keleti Beach Resort
19,Ponta Do Sol,PT,65.03,clear sky,32.6667,-17.1000,Hotel do Campo
20,Buchanan,LR,78.76,scattered clouds,5.8808,-10.0467,Kamaneahn Hotel Inc


In [58]:
# 8a. Create the output File (CSV)
output_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_file, index_label="City_ID")

In [61]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [62]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))